# Importing libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "cache/models--google--codegemma-7b-it/snapshots/078cdc51070553d1636d645c9a238f3b0914459a/"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# I can decide which GPU to use on this node on Leonardo.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Wed Dec 11 14:55:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   43C    P0               81W / 484W|  16523MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Generate the NS3 code for the smart_agriculture scenario.

In [4]:
with open("projects/testbed-at-JNCA/blueprint/sample_blueprint.json") as f:
        sample_blueprint = f.read()
with open("projects/testbed-at-JNCA/blueprint/sample_ns3_code.cc") as f:
        sample_ns3_code = f.read()
with open("projects/testbed-at-JNCA/blueprint/smart_agriculture_complete.json") as f:
        test_blueprint = f.read()

In [5]:
prompt = f"""
        Given the following TEST BLUEPRINT provide me the corresponding NS-3 code. Use the follwing SAMPLE NS-3 code and SAMPLE BLUEPRINT as reference.
        Not add description, just return cpp code.
                TEST BLUEPRINT:

                {test_blueprint}

                SAMPLE BLUEPRINT:

                {sample_blueprint}

                SAMPLE NS3 CODE:

                {sample_ns3_code}


                NS3 CODE:

                """

In [6]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:2134: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


<bos>
        Given the following TEST BLUEPRINT provide me the corresponding NS-3 code. Use the follwing SAMPLE NS-3 code and SAMPLE BLUEPRINT as reference.
        Not add description, just return cpp code.
                TEST BLUEPRINT:

                {
  "name": "Smart Agriculture",
  "description": "A network blueprint for smart agriculture with various sensors and actuators.",
  "experiment_duration":"1 hour",
  "components":{
    "lora_gateways": [
      {
        "name": "GW-RAK-ROOF",
        "description": "Gateway Building 9",
        "latitude": 38.10351066811096,
        "longitude": 13.3459399220741,
        "altitude": 30,
        "gateway_id":"05b0da50148fd6b1"
      }    
    ],
    "lora_devices": [
      {
        "type": "Farm Management System",
        "name":"FMS-001",
        "location": "Farm Office",      
        "responsibilities": ["Crop Monitoring", "Irrigation Control", "Data Analysis"],
        "dev_eui":"629b560f15ed4565",
        "latitude": 38.1086

## Save the .cc code into a file.

In [7]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codeGemma/smart_agriculture.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_city scenario.

In [8]:
with open("projects/testbed-at-JNCA/blueprint/smart_city_complete.json") as f:
        test_blueprint = f.read()

In [9]:
prompt = f"""
        Given the following TEST BLUEPRINT provide me the corresponding NS-3 code. Use the follwing SAMPLE NS-3 code and SAMPLE BLUEPRINT as reference.
        Not add description, just return cpp code.
                TEST BLUEPRINT:

                {test_blueprint}

                SAMPLE BLUEPRINT:

                {sample_blueprint}

                SAMPLE NS3 CODE:

                {sample_ns3_code}


                NS3 CODE:

                """

In [10]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

<bos>
        Given the following TEST BLUEPRINT provide me the corresponding NS-3 code. Use the follwing SAMPLE NS-3 code and SAMPLE BLUEPRINT as reference.
        Not add description, just return cpp code.
                TEST BLUEPRINT:

                {
  "name": "Small Smart City",
    "description": "A network blueprint for a small smart city with various IoT applications.",
    "experiment_duration":"1 hour",

  "components":{
    "lora_gateways": [
      {
        "name": "GW-001",
        "description": "Gateway City Hall",
        "latitude": 38.10351066811096,
        "longitude": 13.3459399220741,
        "gateway_id":"05b0da50148fd6b1",
        "protocol": "LoRaWAN",        
        "application": "LoRa Gateway",
        "location": "City Hall"     
      }    
    ],
    "lora_devices": [
      {
        "dev_eui":"81a63bce0927e700",
        "name":"IOT-001",      
        "application_key":"4ba2162e111f9c2c216b7d68e09f43a8",
        "type": "IoT Sensor",
        "appli

## Save the .cc code into a file.

In [11]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codeGemma/smart_city.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_home scenario.

In [12]:
with open("projects/testbed-at-JNCA/blueprint/smart_home_complete.json") as f:
        test_blueprint = f.read()

In [13]:
prompt = f"""
        Given the following TEST BLUEPRINT provide me the corresponding NS-3 code. Use the follwing SAMPLE NS-3 code and SAMPLE BLUEPRINT as reference.
        Not add description, just return cpp code.
                TEST BLUEPRINT:

                {test_blueprint}

                SAMPLE BLUEPRINT:

                {sample_blueprint}

                SAMPLE NS3 CODE:

                {sample_ns3_code}


                NS3 CODE:

                """

In [14]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

<bos>
        Given the following TEST BLUEPRINT provide me the corresponding NS-3 code. Use the follwing SAMPLE NS-3 code and SAMPLE BLUEPRINT as reference.
        Not add description, just return cpp code.
                TEST BLUEPRINT:

                {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "experiment_duration":"1 hour",
  "wifi_ap":
  [
    {
      "id":"AP-001",
      "ssid":"SmartHomeAPMyExperiment",
      "wpa_passphrase":12345678,
      "wpa_key_mgmt":"WPA-PSK",
      "wlan_IP":"192.168.1.1",
      "eth_IP":"10.8.8.16"
    }
  ],
  "wifi_stations":[
    {
      "id":"ST-001",
      "ssid":"SmartHomeAPMyExperiment",
      "activity": "UDP trasmission at maximum throughput of 30 minutes, start at 0",
      "wpa_passphrase":12345678,
      "wpa_key_mgmt":"WPA-PSK",
      "wlan_IP":"192.168.1.103",
      "wlan_MAC_ADDR":"61:5F:64:5E:90:EB",
      "distance":"10 meter far from AP",
      "eth_IP":"10.8.

## Save the .cc code into a file.

In [15]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codeGemma/smart_home.cc", "w") as file:
    file.write(s)